In [15]:
import numpy
import pandas as pd
dataframe = pd.read_csv("tese-regression-model-v1\dataset_final.csv")
dataset = dataframe.values

#dataframe

In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

X = dataset[:,35:39]
y = dataset[:,0:35]

poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X) #we now have a feature vector with 15 rows instead of only 4

standard_scaler = StandardScaler()
x_scaled = standard_scaler.fit_transform(X_poly)

scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y)

In [17]:
import keras.backend as K
import tensorflow as tf

def custom_loss(y_true, y_pred):
    
    percentage = 0.3
    
    pred_class = y_pred[..., 0:2]
    true_class = tf.argmax(y_true[..., 0:2], -1)
    loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_class, logits=pred_class)
    
    loss_reg1 = K.mean(K.square(y_pred[...,2:17] - y_true[...,2:17]), axis=-1)
    loss_reg2 = K.mean(K.square(y_pred[...,17:35] - y_true[...,17:35]), axis=-1)
    
    loss = percentage*loss_class + (1-percentage)*(loss_reg1*y_true[..., 0] + loss_reg2*y_true[..., 1])
    
    return loss

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense    
from keras import regularizers

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(15, input_dim=15, kernel_initializer='normal',  activation='relu'))
    model.add(Dense(35, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=custom_loss, optimizer='adam') 
    
    return model

In [19]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime

X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled,
                                                    test_size = 0.2,
                                                    random_state = seed)

model = baseline_model()

#Starting Time
start_time = datetime.now()
print('Start Time:', start_time)

history = model.fit(x_scaled, 
                    y_scaled, 
                    validation_split = 0.33,
                    epochs = 500, 
                    batch_size= 128, 
                    verbose = 0)

stop_time = datetime.now()

elapsed_time = stop_time - start_time
print('Elapsed Time:', elapsed_time) 

Start Time: 2018-02-22 22:51:40.257173
Elapsed Time: 0:00:34.328197


In [23]:
y_proba = model.predict(x_scaled)
predicted_inverse = scaler.inverse_transform(y_proba)
print(y[2000])
print(predicted_inverse[2000])
print(y_proba[2000])

[  0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.00000000e-06   6.99000000e-05   2.48000000e-05
   3.70000000e-06   6.40000000e-06   9.59000000e-05   5.00000000e+00
   7.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   3.00000000e+00   8.90000000e-07   7.70000000e-07   7.80000000e-07
   8.90000000e-07   3.00000000e-07   5.50000000e-07]
[ -7.12387991e+00   6.82507944e+00   1.80346506e-05   3.39156795e-05
   1.22853480e-05  -6.93475522e-05   1.71631109e-06   2.90794310e-06
  -7.30882130e-06  -1.00329100e-06   4.00879717e+00   1.10588169e+01
   2.03056788e+00   3.90773315e+01  -4.85577164e+01  -5.16889741e-05
   1.64866371e+01   1.00528484e-06   7.09566593e-05   2.30305977e-05
   1.26298246e-05   6.19859247e-06   7.95038213e-0

In [ ]:
y_classes = y_proba[...,:2].argmax(axis=-1)
print(y_classes)

In [ ]:
numpy.set_printoptions(threshold=numpy.nan)
sess = tf.InteractiveSession()
a = tf.nn.softmax(logits=y_proba[...,:2], dim=-1)
a.eval()

In [ ]:
print(y[730])
print(y_proba[730])